In [3]:
import tensorflow as tf

In [4]:
#显式创建一个graph对象
graph = tf.Graph()

In [11]:
with graph.as_default():
    with tf.name_scope("variables"):
        #跟踪数据流图云心次数的variable对象
        global_step = tf.Variable(0,dtype=tf.float32,trainable=False,name="global_step")
        
        #追踪所有输出随时间的累加和和variable对象
        total_output = tf.Variable(0.0,dtype=tf.float32,trainable=False,name="total_output")
        
    #主要的变换op
    with tf.name_scope("transformation"):
        #独立的输入层
        with tf.name_scope("input"):
            #创建可接收一个向量的占位符
            a = tf.placeholder(tf.float32,shape=[None],name="input_placeholder_a")
            
        #独立的中间层
        with tf.name_scope("intermediate_layer"):
            b = tf.reduce_prod(a,name="product_b")
            c = tf.reduce_sum(a,name="sum_c")
            
        #独立的输出层
        with tf.name_scope("output"):
            output = tf.add(b,c,name="output")
    
    with tf.name_scope("update"):
        #用最新的输出更新variable对象total_output
        update_total = total_output.assign_add(output)
        
        #将最前面的variable对象global_step增1，只要数据流图运行，该操作便需要进行
        increment_step = global_step.assign_add(1)
        
    #汇总op
    with tf.name_scope("summaries"):
        avg = tf.div(update_total,tf.cast(increment_step,tf.float32),name="average")
        
        #创建输出节点汇总数据
        tf.summary.scalar(tensor=output,name="output_summary")
        tf.summary.scalar(tensor=update_total,name="total_summary")
        tf.summary.scalar(tensor=avg,name="avergae_summary")
        
        #全局variable对象和op
        with tf.name_scope("global_ops"):
            #初始化op
            init = tf.global_variables_initializer()
            #将所有汇总数据合并到一个op中
            merged_summaries = tf.summary.merge_all()
    #用明确创建的graph对象启动回话
    sess = tf.Session(graph=graph)
    
    #开启一个summarywrite对象，保存汇总数据
    writer = tf.summary.FileWriter('./improved_graph',graph)
    
    #初始化variable对象
    sess.run(init)
    
    def run_graph(input_tensor):
        feed_dict = {a:input_tensor}
        _,step,summary = sess.run([output,increment_step,merged_summaries],feed_dict=feed_dict)
        
        writer.add_summary(summary,global_step=step)

In [12]:
# 用不同数据流运行该数据流图
run_graph([2,8])
run_graph([3,1,3,3])
run_graph([8])
run_graph([1,2,3])
run_graph([11,4])
run_graph([4,1])
run_graph([7,3,1])
run_graph([6,3])
run_graph([0,2])
run_graph([4,5,6])

In [13]:
#将数据汇总写入磁盘
writer.flush()

#关闭summarywrite对象
writer.close()

#关闭session对象
sess.close()